<a href="https://colab.research.google.com/github/blemke/HyperpameterTuning/blob/master/Copy_of_HyperparameterExercise_CreativeDestruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Hyperparameter tuning using Tensorflow and keras-tuner

In [0]:
!pip uninstall tensorflow

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0


In [0]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 104kB/s 
     |████████████████████████████████| 450kB 33.1MB/s 
     |████████████████████████████████| 3.8MB 41.0MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist



In [0]:
import tensorflow as tf
#print(tf.VERSION)
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.0.0


In [0]:
!pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [0]:
from kerastuner import RandomSearch

In [0]:
TRIALS = 3  # number of models to train
EPOCHS = 2  # number of epoch per model

In [0]:
# Get the MNIST dataset.
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train = np.expand_dims(x_train.astype('float32') / 255, -1)
x_val = np.expand_dims(x_val.astype('float32') / 255, -1)
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)


In [0]:
def build_model(hp):
    """Function that build a TF model based on hyperparameters values.
    Args:
        hp (HyperParameter): hyperparameters values
    Returns:
        Model: Compiled model
    """

    num_layers = hp.Int('num_layers', 2, 8, default=6)
    lr = hp.Choice('learning_rate', [1e-3, 5e-4])

    inputs = layers.Input(shape=(28, 28, 1))
    x = inputs

    for idx in range(num_layers):
        idx = str(idx)

        filters = hp.Int('filters_' + idx, 32, 256, step=32, default=64)
        x = layers.Conv2D(filters=filters, kernel_size=3, padding='same',
                          activation='relu')(x)

        # add a pooling layers if needed
        if x.shape[1] >= 8:
            pool_type = hp.Choice('pool_' + idx, values=['max', 'avg'])
            if pool_type == 'max':
                x = layers.MaxPooling2D(2)(x)
            elif pool_type == 'avg':
                x = layers.AveragePooling2D(2)(x)

    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    # Build model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [0]:
# Initialize the tuner by passing the `build_model` function
# and specifying key search constraints: maximize val_acc (objective),
# and the number of trials to do. More efficient tuners like UltraBand() can
# be used.
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=TRIALS,
                     project_name='hello_world_tutorial_results')

INFO:tensorflow:Reloading Oracle from existing project ./hello_world_tutorial_results/oracle.json


In [0]:
# Display search space overview
tuner.search_space_summary()

In [0]:
# Perform the model search. The search function has the same signature
# as `model.fit()`.
tuner.search(x_train, y_train, batch_size=128, epochs=2,
             validation_data=(x_val, y_val))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 488s 8ms/sample - loss: 0.1437 - accuracy: 0.9553 - val_loss: 0.0418 - val_accuracy: 0.9858
Epoch 2/2
60000/60000 [==============================] - 485s 8ms/sample - loss: 0.0390 - accuracy: 0.9880 - val_loss: 0.0345 - val_accuracy: 0.9889


Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 425s 7ms/sample - loss: 0.2143 - accuracy: 0.9354 - val_loss: 0.0569 - val_accuracy: 0.9803
Epoch 2/2
60000/60000 [==============================] - 425s 7ms/sample - loss: 0.0532 - accuracy: 0.9834 - val_loss: 0.0375 - val_accuracy: 0.9885


Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 700s 12ms/sample - loss: 0.1964 - accuracy: 0.9353 - val_loss: 0.0669 - val_accuracy: 0.9783
Epoch 2/2
60000/60000 [==============================] - 698s 12ms/sample - loss: 0.0522 - accuracy: 0.9843 - val_loss: 0.0396 - val_accuracy: 0.9882


INFO:tensorflow:Oracle triggered exit


In [0]:
# Display the best models, their hyperparameters, and the resulting metrics.
tuner.results_summary()

In [0]:
# Retrieve the best model and display its architecture
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 96)        960       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 192)       166080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 192)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 192)         331968    
_________________________________________________________________
flatten (Flatten)            (None, 9408)              0     